In [1]:
from patchify import patchify
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps
from skimage.transform import resize
import os
from tensorflow.keras.models import load_model, Model
from tqdm import tqdm
import cv2 as cv
import shutil

In [2]:
input_path = r"C:\Users\IA\Documents\Python Scripts\Image to Split\images"
output_path = r"C:\Users\IA\Documents\Python Scripts\Image to Split\patches\\"
model_path = r"..\model\\sinete_model_2.h5"
patch_shape = (256,256,3)
num = 0

model = load_model(model_path)

In [3]:
def is_not_white(image):

    mean_pxl = np.mean(image)
    
    if mean_pxl < 230:
        return True
    else:
        return False

In [4]:
def get_img_array(img_path, size):

    img = tf.keras.utils.load_img(img_path, target_size=size)
    
    array = tf.keras.utils.img_to_array(img)
    
    array = np.expand_dims(array, axis=0)
    return array

In [6]:
n_img = 0
for file in os.listdir(input_path):
    
    # Caminho da imagem
    file_path = os.path.join(input_path, file)
    # Abre a imagem
    #image = Image.open(file_path)
    image = tf.keras.utils.load_img(file_path)
    # Transforma em array
    image = np.asarray(image)
    
    # Extrai os patches da imagem com sobreposição de 50%
    patches = patchify(image, patch_shape, 128)
    
    # Extrai apenas imagens classificadas como anel de sinete
    for i in tqdm(range(patches.shape[0])):
        for j in range(patches.shape[1]):
            n_img += 1
            
            # Carrega patch (256, 256, 3)
            patch = patches[i, j, 0]
            
            # Verifica se o patch não é uma imagem majoritariamente branca
            if is_not_white(patch):
                # Transforma de array para imagem
                img = Image.fromarray(patch)
                # Salva patch
                patch_path = f"{output_path}negativos\patch_{n_img}.png"
                img.save(patch_path)
                # Carrega imagem fazendo o resize (1, 224, 224, 3)
                img_input = get_img_array(patch_path, size=(224,224))
                
                # Classifica imagem
                pred = model.predict(img_input, verbose=0)
                #print(f"{n_img} - {round(pred[0][0]*100, 2)}%")
                
                # Move a imagem se a predição for maior que 95%
                if(pred[0][0] > 0.75):
                    shutil.move(patch_path, f"{output_path}patch_{n_img}.png")

100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [06:15<00:00,  5.96s/it]
